In [ ]:
import tensorflow as tf
import os
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from PIL import Image

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Set the paths to the directories containing the training and validation datasets on Google Drive
train_dir =     '/content/drive/MyDrive/coconut dataset/train',
validation_dir = '/content/drive/MyDrive/coconut dataset/valid'

In [ ]:
# Set the batch size and input shape for the MobileNet model
batch_size = 32
input_shape = (224, 224, 3)

In [ ]:
# Define an ImageDataGenerator for data augmentation on the training data
train_datagen = ImageDataGenerator(
    rescale=1./255)

In [ ]:
# Define an ImageDataGenerator for scaling the validation data
validation_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/coconut dataset/train',
    target_size = (256,256),
    batch_size = 32,
    class_mode = 'categorical'
)

Found 2301 images belonging to 3 classes.


In [ ]:
validation_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/coconut dataset/valid',
    target_size = (256,256),
    batch_size = 64,
    class_mode = 'categorical'
)

Found 531 images belonging to 3 classes.


In [ ]:
# Build the MobileNet model
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=input_shape)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

17225924/17225924 [==============================] - 0s 0us/step


In [ ]:
# Freeze the weights of the base model
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Compile the model
model.compile(optimizer=Adam(lr=0.1), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=5,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

<ipython-input-12-86238129debb>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/5
71/71 [==============================] - ETA: 0s - loss: 0.5701 - accuracy: 0.8052 

71/71 [==============================] - 1121s 16s/step - loss: 0.5701 - accuracy: 0.8052 - val_loss: 0.2318 - val_accuracy: 0.9379
Epoch 2/5
71/71 [==============================] - 439s 6s/step - loss: 0.1818 - accuracy: 0.9480
Epoch 3/5
71/71 [==============================] - 431s 6s/step - loss: 0.1745 - accuracy: 0.9427
Epoch 4/5
71/71 [==============================] - 431s 6s/step - loss: 0.1428 - accuracy: 0.9480
Epoch 5/5
71/71 [==============================] - 424s 6s/step - loss: 0.1654 - accuracy: 0.9449


In [ ]:
# Evaluate the model on the validation set
loss, accuracy = model.evaluate_generator(validation_generator, validation_generator.samples // batch_size)
print('Validation loss:', loss)
print('Validation accuracy:', accuracy)

<ipython-input-13-0985242c16d6>:2: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  loss, accuracy = model.evaluate_generator(validation_generator, validation_generator.samples // batch_size)


Validation loss: 0.20120824873447418
Validation accuracy: 0.9190207123756409


In [ ]:
# Predict on a test image
test_image_path = '/content/drive/MyDrive/coconut dataset/valid/grade3/grade3_rotate206.jpg'
test_image = Image.open(test_image_path).resize(input_shape[:2])
test_image = np.asarray(test_image) / 255.0
test_image = np.expand_dims(test_image, axis=0)
predictions = model.predict(test_image)
predicted_class = np.argmax(predictions)
predicted_prob = predictions[0][predicted_class]

# Print the predicted class and probability
print('Predicted class:', predicted_class)
print('Predicted probability:', predicted_prob)

In [ ]:
model.save("/content/drive/MyDrive/project_mobilenet_dlmodel.h5")

In [ ]:
!pip install gradio


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 63.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.2/286.2 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.7/129.7 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.5/140.5 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 

In [ ]:
import gradio as gr
from tensorflow.keras.models import load_model
import numpy as np

# Load the trained MobileNet model
model = load_model('/content/drive/MyDrive/project_mobilenet_dlmodel.h5')

# Define the prediction function
def predict(image):
    # Preprocess the image
    image = image.reshape((1,) + image.shape)  # Add batch dimension
    image = image / 255.0  # Normalize pixel values
    # Pass the preprocessed image to the model for prediction
    prediction = model.predict(image)
    # Map class indices to grade labels
    grade_labels = ['grade1', 'grade2', 'grade3']
    prediction_grade = [{'grade': label, 'probability': float(prob)} for label, prob in zip(grade_labels, prediction[0])]
    return {'prediction': prediction_grade}

# Set up the Gradio interface
input_image = gr.inputs.Image(shape=(224, 224))
output_prediction = gr.outputs.JSON()

# Create the Gradio interface
gr.Interface(fn=predict, inputs=input_image, outputs=output_prediction, title='MobileNet Image Classification').launch(share=True)


/usr/local/lib/python3.9/dist-packages/gradio/inputs.py:257: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.9/dist-packages/gradio/outputs.py:257: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://87156fb5085ff01b7e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
